In [1]:
from tensorflow import keras
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
num_classes = 5
img_rows,img_cols = 48 , 48
batch_size = 32

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_data_dir = r"/content/drive/MyDrive/facial-expression-recognition-with-keras/face-expression-recognition-dataset/train" #Train images path here
validation_data_dir = r"/content/drive/MyDrive/facial-expression-recognition-with-keras/face-expression-recognition-dataset/validation" #Test images path here

In [5]:
train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 30,
                                    shear_range = 0.3,
                                    zoom_range = 3.0,
                                    width_shift_range = 0.4,
                                    height_shift_range = 0.4,
                                    horizontal_flip = True)
# since my dataset not enough (too small) for training I'm making it different images from this by rotating
# the image by 30% and changing the shear angle and zooming range also shifting the wight and height while
# retaining the dimensions and also by producing mirror images.

In [6]:
validation_data_gen = ImageDataGenerator(rescale = 1./255)

In [7]:
train_generator = train_data_gen.flow_from_directory(train_data_dir,
                                                     color_mode = "grayscale",
                                                     target_size = (img_cols,img_rows),
                                                     batch_size = batch_size,
                                                     class_mode = "categorical",
                                                     shuffle = True)

Found 24322 images belonging to 5 classes.


In [8]:
validation_generator = validation_data_gen.flow_from_directory(validation_data_dir,
                                                               color_mode = "grayscale",
                                                               target_size = (img_cols,img_rows),
                                                               batch_size = batch_size,
                                                               class_mode = "categorical",
                                                               shuffle = True)

Found 5957 images belonging to 5 classes.


In [9]:
model = Sequential() # using sequential modeling

#block1
model.add(Conv2D(32,(3,3),padding="same",
                 kernel_initializer ="he_normal",input_shape=(img_rows,img_cols,1)))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(32,(3,3),padding="same",
                 kernel_initializer ="he_normal",input_shape=(img_rows,img_cols,1)))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [10]:
#block2
model.add(Conv2D(64,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(64,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [11]:
#block3
model.add(Conv2D(128,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(128,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [12]:
#block4
model.add(Conv2D(256,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(256,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [13]:
#block5
model.add(Conv2D(512,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(Conv2D(512,(3,3),padding="same",kernel_initializer ="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [14]:
#block6
model.add(Flatten())
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [15]:
#block7
model.add(Dense(64,kernel_initializer="he_normal"))
model.add(Activation("elu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [16]:
model.add(Dense(num_classes,kernel_initializer="he_normal"))
model.add(Activation("softmax"))

In [17]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [18]:
checkpoint = keras.callbacks.ModelCheckpoint(r"/content/drive/MyDrive/facial-expression-recognition-with-keras/Emotion_little_vgg.h5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1) # checkpoint file name here in my case Emotion_little_vgg.h5

# Here we will monitor the val_loss which is minimum and saving it

earlystop = keras.callbacks.EarlyStopping(monitor="val_loss",
                        min_delta=0,
                        patience=3,
                        restore_best_weights=True,
                        verbose=1)

# Here we will monitor the val_loss whith 3 extra epoches and retain the best weighted file

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=3,
                             verbose=1,
                             min_delta=0.0001)

In [19]:
callbacks=[earlystop,checkpoint,reduce_lr]

model.compile(loss="categorical_crossentropy",
             optimizer=keras.optimizers.Adam(learning_rate=0.001),
             metrics=["accuracy"] )

nb_train_samples = 24322 #obtained after train and validation generators
nb_validation_samples = 5957 #obtained after train and validation generators
epochs = 25

In [20]:
#fitting/training our model
history = model.fit(train_generator,
                    steps_per_epoch=nb_train_samples//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
760/760 [==============================] - ETA: 0s - loss: 1.8309 - accuracy: 0.2360
Epoch 1: val_loss improved from inf to 1.53764, saving model to /content/drive/MyDrive/facial-expression-recognition-with-keras/Emotion_little_vgg.h5
760/760 [==============================] - 4924s 6s/step - loss: 1.8309 - accuracy: 0.2360 - val_loss: 1.5376 - val_accuracy: 0.3068 - lr: 0.0010
Epoch 2/25
760/760 [==============================] - ETA: 0s - loss: 1.5841 - accuracy: 0.2736
Epoch 2: val_loss did not improve from 1.53764
760/760 [==============================] - 975s 1s/step - loss: 1.5841 - accuracy: 0.2736 - val_loss: 1.5464 - val_accuracy: 0.3101 - lr: 0.0010
Epoch 3/25
760/760 [==============================] - ETA: 0s - loss: 1.5616 - accuracy: 0.2942
Epoch 3: val_loss did not improve from 1.53764
760/760 [==============================] - 972s 1s/step - loss: 1.5616 - accuracy: 0.2942 - val_loss: 1.5412 - val_accuracy: 0.3149 - lr: 0.0010
Epoch 4/25
760/760 [============